In [ ]:
!pip install transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# load model
repo_id = "SakanaAI/EvoLLM-JP-v1-7B"
model = AutoModelForCausalLM.from_pretrained(
    repo_id,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto"
    )
tokenizer = AutoTokenizer.from_pretrained(repo_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# prepare inputs
%%time
text = "概要:このタスクでは、商品名から不要なワードを除去する必要があります。不要なワードとは、プロモーションに関連する言葉 や、商品名には不適切な追加情報(例:「送料無料」、「特価」、【中古】、「バナジウム」などの成分、「防災」、「備蓄」など)を指します。以下に具体的な例を示します。例:入力: \"新鮮なトマト 500g 特 価\" 除去後: \"新鮮なトマト 500g\"入力: \"高品質な日本製ノートPC 最新モデル 送料無料\" 除去後: \"高品質な日本製ノートPC 最新モデル\" 入力: \"エコフレンドリー再生紙 A4 500枚セット 大量購入割引\" 除去後: \"エコフレンドリー再生紙 A4 500枚セット\"タスク:以下の商品名から 不要ワードを除去してください。入力：い・ろ・は・す ラベルレス(560ml*48本セット)【いろはす(I LOHAS)】[水 ミネラルウォーター]"
messages = [
    {"role": "system", "content": "あなたはECサイトの経営者でショップの商品について詳しい人物です。"},
    {"role": "user", "content": text},
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")

# generate
from transformers import TextStreamer
output_ids = model.generate(
    inputs.to(model.device),
    pad_token_id=tokenizer.eos_token_id,
    max_new_tokens=768,
    temperature=0.5,
    repetition_penalty=1.15,
    top_p=0.95,
    do_sample=True,
    streamer=TextStreamer(tokenizer, skip_prompt=True),
)

除去後: い・ろ・は・す ラベルレス(560ml*48本セット)</s>
CPU times: user 14.9 s, sys: 314 ms, total: 15.2 s
Wall time: 15.9 s
